In [2]:
!pip install tensorflow

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import os

In [5]:
train_images_dir = 'E://Purni files//AI ML Bootcamp//Class 2 Logistic Regression//train'
test_images_dir = 'E://Purni files//AI ML Bootcamp//Class 2 Logistic Regression//test//test'
labels = pd.read_csv("E://Purni files//AI ML Bootcamp//Class 2 Logistic Regression//trainLabels.csv")

In [6]:
label_map = {label: idx for idx, label in enumerate(labels['label'].unique())}
labels['label'] = labels['label'].map(label_map)

In [7]:
def load_images(image_dir, image_size=(32, 32)):
    image_paths = [os.path.join(image_dir, img_file) for img_file in os.listdir(image_dir)]
    
    def _load_and_preprocess_image(image_path):
        img = tf.io.read_file(image_path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.resize(img, image_size)
        img = img / 255.0
        return img

    images = [tf.numpy_function(_load_and_preprocess_image, [path], tf.float32) for path in image_paths]
    images = tf.stack(images)
    
    image_ids = [int(os.path.splitext(os.path.basename(path))[0]) for path in image_paths]
    image_ids = np.array(image_ids)

    return images, image_ids


In [ ]:
# img and label load
train_images, train_image_ids = load_images(train_images_dir)
test_images, test_image_ids = load_images(test_images_dir)

In [13]:
# Joining labels using the 'id' column
train_data = pd.DataFrame({'id': train_image_ids, 'image': list(train_images.numpy())})
train_data = pd.merge(train_data, labels, on='id')

In [14]:
#train_set
X_train = np.array(train_data['image'].tolist())
y_train = np.array(train_data['label'])

In [15]:
# One-hot encoding
y_train_encoded = to_categorical(y_train)

In [16]:
#train-test split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_encoded, test_size=0.2, random_state=42)

In [17]:
model = Sequential([
    Flatten(input_shape=(32, 32, 3)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  
])


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
# model compilation
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
# model training
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=64)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.2699 - loss: 2.0117 - val_accuracy: 0.3309 - val_loss: 1.8385
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3671 - loss: 1.7707 - val_accuracy: 0.3975 - val_loss: 1.6920
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4047 - loss: 1.6662 - val_accuracy: 0.4176 - val_loss: 1.6447
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4227 - loss: 1.6127 - val_accuracy: 0.4415 - val_loss: 1.5862
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4344 - loss: 1.5792 - val_accuracy: 0.4489 - val_loss: 1.5592
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4493 - loss: 1.5372 - val_accuracy: 0.4520 - val_loss: 1.5610
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4541 - loss: 1.5199 - val_accuracy: 0.4519 - val_loss: 1.5503
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4677 - loss: 1.4907 - val_accuracy: 0.

In [20]:
# test data pediction
test_images_normalized = test_images / 255.0
y_pred = model.predict(test_images_normalized)
y_pred_labels = np.argmax(y_pred, axis=1)

9375/9375 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step


In [21]:
# saving the predictions in a csv file 
predictions = pd.DataFrame({'id': test_image_ids, 'label': y_pred_labels})
predictions.to_csv(r"C://Users//ishti//Downloads//cifar-10//predictions.csv", index=False)

print("Predictions saved to predictions.csv.")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/ishti/Downloads/cifar-10/predictions.csv'

In [26]:
# validation set evaluation
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4529 - loss: 1.5229
Validation Accuracy: 0.45489999651908875


In [28]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Predict on validation data
y_val_pred = np.argmax(model.predict(X_val), axis=1)
y_val_true = np.argmax(y_val, axis=1)

# Calculate performance metrics
accuracy = np.mean(y_val_pred == y_val_true)
precision = precision_score(y_val_true, y_val_pred, average='macro')
recall = recall_score(y_val_true, y_val_pred, average='macro')
f1 = f1_score(y_val_true, y_val_pred, average='macro')
conf_matrix = confusion_matrix(y_val_true, y_val_pred)
class_report = classification_report(y_val_true, y_val_pred)

# Print performance metrics
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Validation Accuracy: 0.4549
Precision: 0.4673
Recall: 0.4535
F1-Score: 0.4471
Confusion Matrix:
[[510  20 110  45 181  30  16 112  15  14]
 [ 17 407   5 328  19  23 108  34  23  51]
 [ 96  27 318  40 247  77  29  56  10  79]
 [ 14  93  14 718  21   5  77  14   5  35]
 [ 88  20 111  60 466  40  24  71  36  75]
 [ 35  59  82  48 121 426  25  73  37  65]
 [ 10  42  18 117  46   3 646  25  10 141]
 [104  34  46  76 165  38  39 366  75  56]
 [109  24  46  61 173  45  37 275 187  24]
 [ 11  48  16  74 108  21 144  23   7 505]]
Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.48      0.50      1053
           1       0.53      0.40      0.46      1015
           2       0.42      0.32      0.36       979
           3       0.46      0.72      0.56       996
           4       0.30      0.47      0.37       991
           5       0.60      0.44      0.51       971
           6       0.56      0